In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit # 可以通过传入一个交叉验证迭代器来使用其他交叉验证策略
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

## 输出导入并分训练集和测试集

In [2]:
iris = datasets.load_iris()
iris.data.shape,iris.target.shape

((150, 4), (150,))

In [3]:
# 我们现在可以快速抽样出训练集测试集，其中40% 作为测试数据
X_train,X_test,y_train,y_test = train_test_split(
    iris.data,iris.target,test_size=0.4,random_state=0)

In [4]:
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((90, 4), (90,), (60, 4), (60,))

In [5]:
clf = svm.SVC(kernel="linear",C=1).fit(X_train,y_train)

In [6]:
clf.score(X_test,y_test)

0.96666666666666667

### 交叉验证

通过将原始数据分为3个数据集合，我们就大大减少了可用于模型学习的样本数量， 并且得到的结果依赖于集合对（训练，验证）的随机选择。

1.将 k-1 份训练集子集作为 training data （训练集）训练模型，
2.将剩余的 1 份训练集子集作为验证集用于模型验证（也就是利用该数据集计算模型的性能指标，例如准确率）。

k-折交叉验证得出的性能指标是循环计算中每个值的平均值。 该方法虽然计算代价很高，但是它不会浪费太多的数据（如固定任意测试集的情况一样）， 在处理样本数据集较少的问题（例如，逆向推理）时比较有优势。

# cross_val_score

1：交叉验证用于评估模型的预测性能，尤其是训练好的模型在新数据上的表现，可以在一定程度上减小过拟合。<br>
2：还可以从有限的数据中获取尽可能多的有效信息。

In [7]:
clf = svm.SVC(kernel="linear",C=1)

In [8]:
scores = cross_val_score(clf, iris.data, iris.target, cv=5)
scores

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

In [9]:
# 评分估计的平均得分和 95% 置信区间由此给出:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.98 (+/- 0.03)


In [10]:
# 通过参数scoring来改变计算方式 (详见相关文档《未完sk3.3》)
# 默认情况，在每一步CV迭代计算的分数是估计量的score方法，可以使用参数scoring改变
scores = cross_val_score(clf, iris.data, iris.target, cv=5, scoring='f1_macro')
scores

array([ 0.96658312,  1.        ,  0.96658312,  0.96658312,  1.        ])

In [11]:
len(scores)

5

In [12]:
n_sapmles = iris.data.shape[0] # 150
cv = ShuffleSplit(n_splits=3,test_size=0.3,random_state=0)
cross_val_score(clf,iris.data,iris.target,cv=cv)

array([ 0.97777778,  0.97777778,  1.        ])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)

In [14]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [15]:
scaler.mean_

array([ 5.83888889,  3.09777778,  3.65111111,  1.15333333])

In [16]:
X_train_transformed = scaler.transform(X_train)
X_test_transformed  = scaler.transform(X_test)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
clf.score(X_test_transformed, y_test)  

0.93333333333333335

In [17]:
# 也可用pipline使代码更简洁

clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, iris.data, iris.target, cv=cv)

array([ 0.97777778,  0.93333333,  0.95555556])

# cross_validate

不同：

它允许指定多个指标进行评估.

除了测试得分之外，它还会返回一个包含训练得分，拟合次数， score-times （得分次数）的一个字典。 It returns a dict containing training scores, fit-times and score-times in addition to the test score.

In [18]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

In [22]:
# 可将多个指标写成list形式
scoring = ["precision_macro" ,"recall_macro"]
clf = svm.SVC(kernel='linear',C=1,random_state=0)
scores = cross_validate(clf, iris.data, iris.target, scoring=scoring, cv=5, return_train_score=False)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [23]:
scores['test_recall_macro']

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

In [24]:
# 默认情况，在每一步CV迭代计算的分数是估计量的score方法，可以使用参数scoring改变
from sklearn.metrics.scorer import make_scorer

In [30]:
scoring = {'prec_macro':'precision_macro',
          'rec_micro': make_scorer(recall_score,average='macro')}
scores = cross_validate(clf,iris.data,iris.target,scoring=scoring,
                       cv=5,return_train_score=True)
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_prec_macro',
 'test_rec_micro',
 'train_prec_macro',
 'train_rec_micro']

In [31]:
scores['train_rec_micro']

array([ 0.975     ,  0.975     ,  0.99166667,  0.98333333,  0.98333333])

获得CV预测
函数cross_val_predict与cross_val_score有相似的接口，但是对于输入的每一个元素，返回的预测是那个元素在检验集获得的。只有分派所有元素到检验集一次的交叉验证能被使用。这些预测能够被用来评价分类器。

In [32]:
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(clf, iris.data, iris.target, cv=10)
metrics.accuracy_score(iris.target, predicted)


0.97333333333333338

# k-fold

KFold分割所有的样本进入相等大小的k组，称folds. 使用k-1 folds学习预测函数，余下的fold用于检验。下面的例子是一个4样本的2-fold交叉验证。


In [33]:
import numpy as np
from sklearn.model_selection import KFold
X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
     print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]


In [34]:
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
